**UNet Resistivity Trial**

Run the cleaned dataset through a UNet model

In [0]:
import pandas as pd

# Inspect dataframe
df = pd.read_csv("/dbfs/FileStore/rachlenn/df_chunks.csv")
display(df.head(10))


In [0]:
import numpy as np

def parse_signal(x):
    # If already list/array, just convert
    if isinstance(x, (list, np.ndarray)):
        return np.array(x, dtype=np.float32)
    # Otherwise assume it's a string like "[ -1.63 -1.53 ... ]"
    x = x.strip().replace("\n", " ")       # remove newlines
    x = x.strip("[]")                      # remove leading/trailing brackets
    return np.array(x.split(), dtype=np.float32)

signals = df["signal_scaled"].apply(parse_signal)

# Stack into 2D numpy array
X = np.stack(signals.to_numpy())
y = df["fish_present"].astype(np.float32).to_numpy()


In [0]:
# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y
)

max_len = 1000

Define 1D U-Net

In [0]:
from tensorflow.keras import layers, models

def unet_1d(input_length, num_classes=1):
    inputs = layers.Input(shape=(input_length, 1))

    # Encoder
    c1 = layers.Conv1D(16, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv1D(16, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling1D(2)(c1)

    c2 = layers.Conv1D(32, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv1D(32, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling1D(2)(c2)

    c3 = layers.Conv1D(64, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv1D(64, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling1D(2)(c3)

    # Bottleneck
    b = layers.Conv1D(128, 3, activation='relu', padding='same')(p3)
    b = layers.Conv1D(128, 3, activation='relu', padding='same')(b)

    # Decoder
    u3 = layers.UpSampling1D(2)(b)
    u3 = layers.Concatenate()([u3, c3])
    c6 = layers.Conv1D(64, 3, activation='relu', padding='same')(u3)

    u2 = layers.UpSampling1D(2)(c6)
    u2 = layers.Concatenate()([u2, c2])
    c7 = layers.Conv1D(32, 3, activation='relu', padding='same')(u2)

    u1 = layers.UpSampling1D(2)(c7)
    u1 = layers.Concatenate()([u1, c1])
    c8 = layers.Conv1D(16, 3, activation='relu', padding='same')(u1)

    # Output (binary classification at sequence level)
    gap = layers.GlobalAveragePooling1D()(c8)
    output = layers.Dense(num_classes, activation='sigmoid')(gap)

    model = models.Model(inputs, output)
    return model

model = unet_1d(max_len)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()])
